<a href="https://colab.research.google.com/github/GabrielMou21/projeto-bd-colab/blob/main/Projeto_BD_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# projeto_bd_sqlite.py
import sqlite3
import pandas as pd

def criar_banco_dados():
    """Cria e configura o banco de dados SQLite"""
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = ON;")
    print("Banco de dados criado com sucesso!")
    return conn, cursor

def criar_tabelas(cursor):
    """Cria todas as tabelas necessárias"""
    tabelas = [
        """
        CREATE TABLE IF NOT EXISTS clientes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            email TEXT UNIQUE,
            data_cadastro DATE DEFAULT CURRENT_DATE
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS produtos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            categoria TEXT,
            preco REAL CHECK(preco > 0),
            estoque INTEGER DEFAULT 0
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS pedidos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cliente_id INTEGER,
            data_pedido TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            status TEXT DEFAULT 'Pendente',
            FOREIGN KEY (cliente_id) REFERENCES clientes(id) ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS itens_pedido (
            pedido_id INTEGER,
            produto_id INTEGER,
            quantidade INTEGER CHECK(quantidade > 0),
            preco_unitario REAL,
            PRIMARY KEY (pedido_id, produto_id),
            FOREIGN KEY (pedido_id) REFERENCES pedidos(id) ON DELETE CASCADE,
            FOREIGN KEY (produto_id) REFERENCES produtos(id)
        )
        """
    ]

    for tabela in tabelas:
        cursor.execute(tabela)
    print("Tabelas criadas com sucesso!")

def popular_dados(cursor):
    """Insere dados iniciais nas tabelas"""
    # Clientes
    clientes = [
        ('Ana Silva', 'ana@email.com'),
        ('Carlos Oliveira', 'carlos@email.com'),
        ('Mariana Souza', 'mariana@email.com'),
        ('João Santos', 'joao@email.com')
    ]
    cursor.executemany("INSERT INTO clientes (nome, email) VALUES (?, ?)", clientes)

    # Produtos
    produtos = [
        ('Notebook Dell', 'Eletrônicos', 4500.00, 10),
        ('Smartphone Samsung', 'Eletrônicos', 2200.00, 15),
        ('Mesa de Escritório', 'Móveis', 800.00, 5),
        ('Cadeira Ergonômica', 'Móveis', 650.00, 8),
        ('Mouse Sem Fio', 'Acessórios', 120.00, 20)
    ]
    cursor.executemany(
        "INSERT INTO produtos (nome, categoria, preco, estoque) VALUES (?, ?, ?, ?)",
        produtos
    )

    # Pedidos
    pedidos = [
        (1, '2023-11-01', 'Pendente'),
        (2, '2023-11-02', 'Processando'),
        (3, '2023-11-02', 'Pendente')
    ]
    cursor.executemany(
        "INSERT INTO pedidos (cliente_id, data_pedido, status) VALUES (?, ?, ?)",
        pedidos
    )

    # Itens de Pedido
    itens = [
        (1, 1, 1, 4500.00),
        (1, 5, 2, 120.00),
        (2, 2, 1, 2200.00),
        (3, 3, 1, 800.00),
        (3, 4, 2, 650.00)
    ]
    cursor.executemany(
        "INSERT INTO itens_pedido VALUES (?, ?, ?, ?)",
        itens
    )

    print("Dados inseridos com sucesso!")

def executar_consultas(cursor):
    """Executa e exibe as consultas principais"""
    consultas = {
        "Clientes": "SELECT * FROM clientes",
        "Produtos com estoque baixo": "SELECT nome, estoque FROM produtos WHERE estoque < 10",
        "Pedidos com clientes": """
            SELECT p.id, c.nome, p.data_pedido, p.status
            FROM pedidos p JOIN clientes c ON p.cliente_id = c.id
        """,
        "Relatório de pedidos": """
            SELECT ped.id, cli.nome, prod.nome, it.quantidade,
                   it.preco_unitario, (it.quantidade * it.preco_unitario) AS subtotal
            FROM pedidos ped
            JOIN clientes cli ON ped.cliente_id = cli.id
            JOIN itens_pedido it ON ped.id = it.pedido_id
            JOIN produtos prod ON it.produto_id = prod.id
            ORDER BY ped.id
        """,
        "Total por pedido": """
            SELECT ped.id, cli.nome, SUM(it.quantidade * it.preco_unitario) AS total
            FROM pedidos ped
            JOIN clientes cli ON ped.cliente_id = cli.id
            JOIN itens_pedido it ON ped.id = it.pedido_id
            GROUP BY ped.id
        """
    }

    for titulo, consulta in consultas.items():
        print(f"\n=== {titulo} ===")
        cursor.execute(consulta)
        df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
        display(df)

def atualizar_estoque(cursor):
    """Atualiza o estoque baseado nos pedidos"""
    cursor.execute("""
        UPDATE produtos
        SET estoque = estoque - (
            SELECT COALESCE(SUM(quantidade), 0)
            FROM itens_pedido
            WHERE produto_id = produtos.id
        )
    """)
    print("\nEstoque atualizado com sucesso!")
    cursor.execute("SELECT nome, estoque FROM produtos")
    df = pd.DataFrame(cursor.fetchall(), columns=['Produto', 'Estoque'])
    display(df)

def main():
    """Função principal que orquestra a execução"""
    conn, cursor = criar_banco_dados()

    try:
        criar_tabelas(cursor)
        popular_dados(cursor)
        conn.commit()

        executar_consultas(cursor)
        atualizar_estoque(cursor)
        conn.commit()

    finally:
        conn.close()
        print("\nConexão com o banco de dados encerrada.")

if __name__ == "__main__":
    main()